## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
0,0,Butaritari,KI,2021-02-17 05:18:34,3.0707,172.7902,81.68,82,0,18.12,clear sky,0,0
1,1,Broome,US,2021-02-17 05:18:34,42.2506,-75.8330,15.80,73,75,17.27,broken clouds,0,0
2,2,Hermanus,ZA,2021-02-17 05:18:34,-34.4187,19.2345,62.69,80,1,5.93,clear sky,0,0
3,3,Punta Arenas,CL,2021-02-17 05:18:35,-53.1500,-70.9167,46.40,81,0,10.36,clear sky,0,0
4,4,Agadir,MA,2021-02-17 05:18:35,30.4202,-9.5982,48.20,93,0,6.91,clear sky,0,0


In [4]:
# 2. Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
# Prompt the customer for the maximum temperature preference. 
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
0,0,Butaritari,KI,2021-02-17 05:18:34,3.0707,172.7902,81.68,82,0,18.12,clear sky,0,0
7,7,Madang,PG,2021-02-17 05:18:35,-5.2246,145.7966,80.15,83,100,7.58,overcast clouds,0,0
10,10,Saint-Joseph,RE,2021-02-17 05:18:36,-21.3667,55.6167,82.00,63,93,2.08,overcast clouds,0,0
12,12,Aden,YE,2021-02-17 05:18:37,12.7794,45.0367,76.60,70,93,15.99,overcast clouds,0,0
19,19,Nyimba,ZM,2021-02-17 05:18:38,-14.5566,30.8149,70.77,92,100,2.06,overcast clouds,0,0
20,20,Saint-Philippe,RE,2021-02-17 05:18:38,-21.3585,55.7679,82.98,61,92,2.75,overcast clouds,0,0
25,25,Rikitea,PF,2021-02-17 05:18:39,-23.1203,-134.9692,78.78,76,27,14.14,light rain,0,0
33,33,Hithadhoo,MV,2021-02-17 05:18:41,-0.6000,73.0833,81.75,70,99,1.83,overcast clouds,0,0
34,34,Touros,BR,2021-02-17 05:18:41,-5.1989,-35.4608,78.80,83,20,4.61,few clouds,0,0
38,38,Busselton,AU,2021-02-17 05:18:41,-33.6500,115.3333,82.99,54,0,5.99,clear sky,0,0


In [6]:
# 4a. Determine if there are any empty rows.
# pulled from https://stackoverflow.com/questions/27159189/find-empty-or-nan-entry-in-pandas-dataframe
np.where(pd.isnull(preferred_cities_df))

(array([166, 171]), array([2, 2]))

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='all')

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.68,clear sky,3.0707,172.7902,
7,Madang,PG,80.15,overcast clouds,-5.2246,145.7966,
10,Saint-Joseph,RE,82.00,overcast clouds,-21.3667,55.6167,
12,Aden,YE,76.60,overcast clouds,12.7794,45.0367,
19,Nyimba,ZM,70.77,overcast clouds,-14.5566,30.8149,
20,Saint-Philippe,RE,82.98,overcast clouds,-21.3585,55.7679,
25,Rikitea,PF,78.78,light rain,-23.1203,-134.9692,
33,Hithadhoo,MV,81.75,overcast clouds,-0.6000,73.0833,
34,Touros,BR,78.80,few clouds,-5.1989,-35.4608,
38,Busselton,AU,82.99,clear sky,-33.6500,115.3333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Print the dataframe
hotel_df = pd.DataFrame(hotel_df)
hotel_df.dropna(how='all')

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.68,clear sky,3.0707,172.7902,Isles Sunset Lodge
7,Madang,PG,80.15,overcast clouds,-5.2246,145.7966,Comfort Inn Hotel
10,Saint-Joseph,RE,82.00,overcast clouds,-21.3667,55.6167,"""Plantation Bed and Breakfast"
12,Aden,YE,76.60,overcast clouds,12.7794,45.0367,Sama Emirate Hotel
19,Nyimba,ZM,70.77,overcast clouds,-14.5566,30.8149,Dali Dali Lodge
...,...,...,...,...,...,...,...
668,Lima,PE,73.00,clear sky,-12.0432,-77.0282,Gran Hotel Bolivar Lima
669,Mahajanga,MG,84.20,few clouds,-15.7167,46.3167,Restaurant Greedy Coco Lodge Majunga
671,San Miguel,SV,74.70,broken clouds,13.4833,-88.1833,Hotel Tropico Inn San Miguel
675,Bende,NG,74.66,broken clouds,5.5572,7.6368,Bende Local Government Corppers Lodge


In [11]:
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.68,clear sky,3.0707,172.7902,Isles Sunset Lodge
7,Madang,PG,80.15,overcast clouds,-5.2246,145.7966,Comfort Inn Hotel
10,Saint-Joseph,RE,82.00,overcast clouds,-21.3667,55.6167,"""Plantation Bed and Breakfast"
12,Aden,YE,76.60,overcast clouds,12.7794,45.0367,Sama Emirate Hotel
19,Nyimba,ZM,70.77,overcast clouds,-14.5566,30.8149,Dali Dali Lodge
...,...,...,...,...,...,...,...
668,Lima,PE,73.00,clear sky,-12.0432,-77.0282,Gran Hotel Bolivar Lima
669,Mahajanga,MG,84.20,few clouds,-15.7167,46.3167,Restaurant Greedy Coco Lodge Majunga
671,San Miguel,SV,74.70,broken clouds,13.4833,-88.1833,Hotel Tropico Inn San Miguel
675,Bende,NG,74.66,broken clouds,5.5572,7.6368,Bende Local Government Corppers Lodge


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

IndexError: list index out of range